In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.add("FFMPEG")
Pkg.instantiate()
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
# import FFMPEG
using LinearAlgebra
using Random
using JLD2
using Test
using StaticArrays
using Printf
using MuJoCo

  Activating project at `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Project.toml`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Project.toml`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Manifest.toml`


In [2]:
# include fmincon for ipopt
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [3]:
"""
x = fmincon(cost,equality_constraint,inequality_constraint,x_l,x_u,c_l,c_u,x0,params,diff_type)

This function uses IPOPT to minimize an objective function 

`cost(params, x)` 

With the following three constraints: 

`equality_constraint(params, x) = 0`
`c_l <= inequality_constraint(params, x) <= c_u` 
`x_l <= x <= x_u` 

Note that the constraint functions should return vectors. 

Problem specific parameters should be loaded into params::NamedTuple (things like 
cost weights, dynamics parameters, etc.). 

args:
    cost::Function                    - objective function to be minimzed (returns scalar)
    equality_constraint::Function     - c_eq(params, x) == 0 
    inequality_constraint::Function   - c_l <= c_ineq(params, x) <= c_u 
    x_l::Vector                       - x_l <= x <= x_u 
    x_u::Vector                       - x_l <= x <= x_u 
    c_l::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    c_u::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    x0::Vector                        - initial guess 
    params::NamedTuple                - problem parameters for use in costs/constraints 
    diff_type::Symbol                 - :auto for ForwardDiff, :finite for FiniteDiff 
    verbose::Bool                     - true for IPOPT output, false for nothing 

optional args:
    tol                               - optimality tolerance 
    c_tol                             - constraint violation tolerance 
    max_iters                         - max iterations 
    verbose                           - verbosity of IPOPT 

outputs:
    x::Vector                         - solution 

You should try and use :auto for your `diff_type` first, and only use :finite if you 
absolutely cannot get ForwardDiff to work. 

This function will run a few basic checks before sending the problem off to IPOPT to 
solve. The outputs of these checks will be reported as the following:

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

If you're getting stuck during the testing of one of the derivatives, try switching 
to FiniteDiff.jl by setting diff_type = :finite. 
""";

In [4]:
# import piano model
model = load_model("cartpole.xml")
data = init_data(model)

MuJoCo Data object

In [5]:

step!(model, data)

println("Initial position: ", data.qpos)
println("Initial velocity: ", data.qvel)
println("model states:",get_physics_state(model, data))

Initial position: [0.0; 0.0;;]
Initial velocity: [0.0; 0.0;;]
model states:[0.0, 0.0, 0.0, 0.0]


## Test DIRCOL: moving index finger left right

In [6]:
# cartpole 
function dynamics(params::NamedTuple, x::Vector, u)
    # cartpole ODE, parametrized by params. 

    # cartpole physical parameters 
    mc, mp, l = params.mc, params.mp, params.l
    g = 9.81
    
    q = x[1:2]
    qd = x[3:4]

    s = sin(q[2])
    c = cos(q[2])

    H = [mc+mp mp*l*c; mp*l*c mp*l^2]
    C = [0 -mp*qd[2]*l*s; 0 0]
    G = [0, mp*g*l*s]
    B = [1, 0]

    qdd = -H\(C*qd + G - B*u[1])
    xdot = [qd;qdd]
    return xdot 

end
function hermite_simpson(params::NamedTuple, x1::Vector, x2::Vector, u, dt::Real)::Vector
    # TODO: input hermite simpson implicit integrator residual 
    ẋ1 = dynamics(params, x1, u)
    ẋ2 = dynamics(params, x2, u)
    xm = (1/2)*(x1 + x2) + (dt/8)*(ẋ1 - ẋ2)
    ẋm = dynamics(params, xm, u)
    x1 + (dt/6)*(ẋ1 + 4*ẋm + ẋ2) - x2
end
     

hermite_simpson (generic function with 1 method)

In [31]:
function wrapped_mj_step(model, data, x, uk, forward)    
    # given the current model and data. set the state and control to the model and perform a forward step
    if typeof(x) == Vector{Float64}
        # set control 
        data.ctrl[:] .= uk
        # set state
        data.qpos .= x[1:model.nq]
        data.qvel .= x[(model.nq + 1):end]
    else
        # if using diff types, we need to convert the dual numbers to floats
        converted_uk = ForwardDiff.value.(uk)
        converted_x = ForwardDiff.value.(x)
        # set control
        data.ctrl[:] .= converted_uk

        # set state
        data.qpos .= converted_x[1:model.nq]
        data.qvel .= converted_x[(model.nq + 1):end]
    end
    
    # take discrete dynamics step 
    if forward
        forward!(model,data)
    else
        step!(model, data) 
    end
    # return updated state k + 1
    xkp1 = zeros(model.nq + model.nv) 
    xkp1 .= get_physics_state(model, data)
    
    return xkp1
end

function cartpole_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg = params.idx, params.N, params.xg
    Q, R, Qf = params.Q, params.R, params.Qf

    J = 0 
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        xkp1 = wrapped_mj_step(model, data, xi, ui,true)
        J += 0.5*(xi-xg)'*Q*(xi-xg)+0.5*ui'*R*ui
    end
    
    # dont forget terminal cost 
    xt = Z[idx.x[N]]
    J += 0.5*(xt-xg)'*Q*(xt-xg)
    return J 
end
function cartpole_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model
    data = init_data(model)
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # TODO: hermite simpson 
        # println("xi: ", xi[1:5])
        # print("ui: ", typeof(ui))
        # print(hermite_simpson(params, xi, xip1, ui, dt))
        xip1_mujoco = wrapped_mj_step(model, data, xi, ui,false)
        c[idx.c[i]] .= xip1_mujoco - xip1
        # print(c[idx.c[i]])
    end
    # println(typeof(c))
    return c 
end

function cartpole_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 

    con_1 = Z[idx.x[1]] - xic
    con_2 = Z[idx.x[N]] - xg

    return [con_1; con_2; cartpole_dynamics_constraints(params, Z)]
    # return [con_1; cartpole_dynamics_constraints(params, Z)]
end
function cartpole_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    return zeros(eltype(Z), 0)
    # return inequality_constraints
end
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

create_idx (generic function with 1 method)

In [32]:

function cartpole_solve_moving(;verbose=true)
    # instantiate model and data
    model = load_model("cartpole.xml")
    data = init_data(model)

    # reset the model and data
    reset!(model, data)
    
    # initiate time and time steps
    dt = 0.1
    tf = 2.0 
    t_vec = 0:dt:tf 
    N = length(t_vec)

    # LQR cost
    # Q size of full states
    # Q = diagm(ones(model.nq + model.nv))
    # R = 0.1*diagm(ones(model.nu))
    # Qf = 10*diagm(ones(model.nq + model.nv))


    # Q size only 3 because we track one finger cartesian coordinates
    Q = 1*diagm(ones(model.nq + model.nv))
    R = 0.1*diagm(ones(model.nu))
    Qf = 100*diagm(ones(model.nq + model.nv))

    # indexing 
    idx = create_idx(model.nq + model.nv, model.nu, N)

    # initial and goal state
    xic = [0, pi, 0.1, 0]
    xg = [0, 0, 0, 0]
    
    params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data,mc = 1.0, mp = 0.2, l = 0.5)


    # primal bounds
    x_l = -1 * Inf * ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    
    x_l = -1*Inf*ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    for i = 1:(N-1)
        x_l[idx.u[i]] .= -0.7
        x_u[idx.u[i]] .= 0.7
    end
    # inequality constraints
    # c_l = -1 * Inf * ones(3*(idx.N-1))
    # c_u = Inf * ones(3*(idx.N-1))
    c_l = zeros(0)
    c_u = zeros(0)
    # initial guess 
    z0 = 1*randn(idx.nz)

    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    # diff_type = :auto 
    diff_type = :finite
    
    
    Z = fmincon(cartpole_cost,cartpole_equality_constraints,cartpole_inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = true)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    return X, U, t_vec, params
end

cartpole_solve_moving (generic function with 1 method)

In [33]:
X, U, t_vec, params = cartpole_solve_moving(verbose=true)

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :finite (FiniteDiff.jl)---
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     9152
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      104
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       20
                     variables with only upper bounds:        0
Total number of equality constraints.................:       88
Total number of inequality constraints.......

([[-1.8358199814864264e-35, 3.141592653589793, 0.1, 0.0], [0.004419050484868124, 3.1289992535678204, -0.01083306231870952, -0.24201641185890063], [-0.08871323613118577, 2.900985994439162, -1.829387813872048, -4.108186113788055], [-0.3638755748552248, 2.356932133393367, -3.6067014924843606, -6.25917450475084], [-0.7989806015765879, 1.7565893804355488, -5.085829505013795, -5.345825240392253], [-1.1976150964787347, 1.3235887594881317, -2.185635313414904, -3.5325598785230805], [-1.3100435461544024, 1.0078178315883755, -0.33774753691000503, -2.800422502842265], [-1.2755997299805368, 0.7327791229765486, 1.0345027714376087, -2.900755554878442], [-1.1496002426614325, 0.4800671160028841, 1.487416509067932, -2.2751935330198103], [-0.9889771926495269, 0.28120896397596595, 1.729278314633505, -1.7987449932806148]  …  [-0.6339496526317847, 0.0056111858476748135, 1.7662997581774316, -1.061370957105805], [-0.4638573093416536, -0.08438498234446341, 1.63918541233366, -0.7872380931595258], [-0.3096309133

In [34]:
X

21-element Vector{Vector{Float64}}:
 [-1.8358199814864264e-35, 3.141592653589793, 0.1, 0.0]
 [0.004419050484868124, 3.1289992535678204, -0.01083306231870952, -0.24201641185890063]
 [-0.08871323613118577, 2.900985994439162, -1.829387813872048, -4.108186113788055]
 [-0.3638755748552248, 2.356932133393367, -3.6067014924843606, -6.25917450475084]
 [-0.7989806015765879, 1.7565893804355488, -5.085829505013795, -5.345825240392253]
 [-1.1976150964787347, 1.3235887594881317, -2.185635313414904, -3.5325598785230805]
 [-1.3100435461544024, 1.0078178315883755, -0.33774753691000503, -2.800422502842265]
 [-1.2755997299805368, 0.7327791229765486, 1.0345027714376087, -2.900755554878442]
 [-1.1496002426614325, 0.4800671160028841, 1.487416509067932, -2.2751935330198103]
 [-0.9889771926495269, 0.28120896397596595, 1.729278314633505, -1.7987449932806148]
 ⋮
 [-0.4638573093416536, -0.08438498234446341, 1.63918541233366, -0.7872380931595258]
 [-0.30963091338274645, -0.1500643830756864, 1.4481785326816796, -

In [67]:
U

20-element Vector{Vector{Float64}}:
 [-0.6265833658723434]
 [-0.7000000099986999]
 [-0.70000000999854]
 [-0.7000000099819587]
 [0.7000000099575433]
 [0.41520785504151847]
 [0.18457504214038417]
 [0.10556871767714979]
 [0.045672185919134425]
 [0.001824803554583497]
 [-0.029825744283374242]
 [-0.052324394053797636]
 [-0.06850530514641762]
 [-0.0807773709461258]
 [-0.09085955277886092]
 [-0.09958205396599723]
 [-0.10661465723769309]
 [-0.10994870572178533]
 [-0.10495161902736722]
 [0.1293425167734886]

In [68]:

Pkg.add("FileIO")
using FileIO, JLD2
FileIO.save("output_States.jld2","x",X)
FileIO.save("output_controls.jld2","u",U)


In [26]:
# instantiate model and data
model = load_model("cartpole.xml")
data = init_data(model)

# reset the model and data
reset!(model, data)

nx = model.nq + model.nv + model.na # State vector dimension
N = length(t_vec)
states = zeros(nx, N)
ctrl_states = zeros(model.nu, N)
N = length(t_vec)
reset!(model, data)
states[:,1] = X[1][:]

data.qpos .= X[1][1:2]
data.qvel .= X[1][3:4]
step!(model, data)

for t in 1:(N-1)
    data.ctrl .= U[t][:]
    states[:,t+1] = X[1][:]
end
print(U)

[[9.327146691651987e-5], [0.000323561397891853], [-4.585920482610556e-5], [0.00010186056392591248], [-0.00024495443791071683], [8.884401970756024e-5], [-8.263861541302976e-5], [-0.00019708036517860443], [-0.0003416861681224594], [-4.127368759019482e-5], [8.306414261238956e-6], [0.0001684958792046587], [-1.5739918117118115e-5], [3.8073070150289074e-5], [-0.00011194767920866136], [2.7999628230439994e-5], [7.48171788077857e-6], [0.00039812296744256914], [-3.4063474725752594e-5], [-8.78932921838786e-5], [-0.00035995865392562986], [9.447557521185401e-6], [-0.0001798659656717273], [0.00013788478252301504], [-4.7121676604250896e-5], [-0.00013595312013107316], [-1.8502288724171114e-5], [-2.3918808539490525e-5], [1.821343691785399e-5], [-0.0001399540397580862], [-3.2790399610036294e-5], [-0.0001665571673643603], [-0.0003080130253928047], [-0.0002067174619381835], [-8.623973367267194e-5], [-0.00013945605744015383], [-4.151876244093657e-5], [3.208788486972738e-5], [5.44054133734033e-5], [-0.00016

In [27]:
init_visualiser()
visualise!(model, data, trajectories = states)

ERROR: LoadError: InitError: could not load library "/home/aipex6/.julia/artifacts/648a32a349aac06f19b6bfed47f0b822b4ef28c3/lib/libgobject-2.0.so"
/home/aipex6/.julia/artifacts/648a32a349aac06f19b6bfed47f0b822b4ef28c3/lib/libgobject-2.0.so: undefined symbol: g_dir_unref
Stacktrace:
  [1] dlopen(s::String, flags::UInt32; throw_error::Bool)
    @ Base.Libc.Libdl ./libdl.jl:117
  [2] dlopen(s::String, flags::UInt32)
    @ Base.Libc.Libdl ./libdl.jl:116
  [3] macro expansion
    @ ~/.julia/packages/JLLWrappers/pG9bm/src/products/library_generators.jl:63 [inlined]
  [4] __init__()
    @ Glib_jll ~/.julia/packages/Glib_jll/ac15p/src/wrappers/x86_64-linux-gnu.jl:36
  [5] run_module_init(mod::Module, i::Int64)
    @ Base ./loading.jl:1134
  [6] register_restored_modules(sv::Core.SimpleVector, pkg::Base.PkgId, path::String)
    @ Base ./loading.jl:1122
  [7] _include_from_serialized(pkg::Base.PkgId, path::String, ocachepath::String, depmods::Vector{Any})
    @ Base ./loading.jl:1067
  [8] _requ

ErrorException: Failed to precompile FFMPEG [c87230d0-a227-11e9-1b43-d7ebe4e7570a] to "/home/aipex6/.julia/compiled/v1.10/FFMPEG/jl_IifO5C".